In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException, JavascriptException
from selenium_stealth import stealth
from bs4 import BeautifulSoup
from utils import build_driver

drv = build_driver()

In [ ]:
from collections import defaultdict
import pandas as pd
import csv
import time
import re
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_explain_link(drv, pathto):
    drv.get(pathto)
    time.sleep(2)

    # 1) 팝업 닫기 (Selenium WebElement로 직접)
    try:
        close_btn = WebDriverWait(drv, 3).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".pop-ad .btn-close"))
        )
        close_btn.click()
        time.sleep(1)  # 닫힌 후 UI 안정화
        print("Exist")
    except (TimeoutException, NoSuchElementException) as e:
        print("No", e)
        pass  # 팝업이 없으면 그냥 넘어감

    # 2) 팝업 닫은 후 HTML 다시 읽기
    html = drv.page_source
    soup = BeautifulSoup(html, "html.parser")

    try:
        # 3) 정상 파싱
        professor_profile = soup.find('div', class_='professor-profile')
        lab_link = professor_profile.find('p', class_='site').find("a")['href']
    
        main = soup.find('ul', class_='open-lab-data-list')
        content = main.find_all('li')[0].find('div', class_='open-lab-content-area')
    
        return lab_link, content
    except Exception as e:
        print("Error ", e)
        return "", ""

datas = []
# 68까지

for i in tqdm(range(1, 200)):
    url = f'https://phdkim.net/professor/open-lab?page={i}'

    try:
        drv.get(url)
        time.sleep(2)
        
        html = drv.page_source
        soup = BeautifulSoup(html, "html.parser")
        
        lab_list = soup.find('div', class_='open-lab-list')
        if not lab_list:
            print("END! At ", i)
            break
        ais = lab_list.find_all('a', class_='clearfix')
        print("Len ais ", len(ais))
        for idx in range(len(ais)):
            dpath = 'https://phdkim.net' + ais[idx]['href']
            lablink, content = get_explain_link(drv, dpath)

            getd = {
                'detail_path': dpath,
                'university': ais[idx].find("p", class_='university').text.strip(),
                'lablink': lablink,
                'content': content
            }
            datas.append(getd)
        
        df = pd.DataFrame(datas)
        df.to_csv("phdlabs2.csv", index=False)
        
        time.sleep(1)
    except Exception as e:
        print(f"Error at {i}th ", e)
        traceback.print_exc()


In [2]:
!python3 -m pip install crawlbase

Using legacy 'setup.py install' for crawlbase, since package 'wheel' is not installed.
    Running setup.py install for crawlbase ... done
You should consider upgrading via the '/Users/gimhojin/Desktop/sonus/myenv/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
from crawlbase import CrawlingAPI
import json

# Initialize Crawlbase API with your access token
crawling_api = CrawlingAPI({ 'token': '' })

URL = 'https://www.linkedin.com/in/hojin-kim-2a553222a/'

options = {
    'scraper': 'linkedin-profile',
    'async': 'true'
}

In [7]:
response = crawling_api.get(URL, options)

In [8]:
response

{'headers': {'original_status': 'None', 'pc_status': '200', 'url': 'None'},
 'status_code': 200,
 'body': b'Please visit the below link to enable LinkedIn crawling. \nhttps://crawlbase.com/dashboard/account/domain/linkedin/agreement \n'}

In [ ]:
if response['status_code'] == 200:
    return json.loads(response['body'].decode('utf-8'))
else:
    print("Failed to fetch the page. Status code:", response['status_code'])
    return None

def scrape_profile(url):
try:
    json_response = make_crawlbase_request(url)
    print("json_response : ", json_response)
    if json_response:
        return json_response
except Exception as e:
    print(f"Request failed: {e}")

return None

if __name__ == '__main__':
scraped_data = scrape_profile(URL)
print(json.dumps(scraped_data, indent=2))

In [4]:
json_response

NameError: name 'json_response' is not defined

In [3]:
!python3 -m pip install huggingface_hub

In [ ]:
huggingface_api_key = ""

def upload_to_huggingface(file_path):
    from huggingface_hub import HfApi
    from huggingface_hub import login

    api = HfApi()
    login(token=huggingface_api_key)


    api.upload_file(
        path_or_fileobj=file_path,
        path_in_repo=file_path.split("/")[-1],
        repo_id="Daniel777/harper",
        repo_type="model",
    )

upload_to_huggingface("./google_serp/linkedins_total_1216.csv")

Processing Files (1 / 1): 100%|██████████| 19.7MB / 19.7MB,  0.00B/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
